In [1]:
!pip install transformers datasets sentencepiece sacrebleu nltk accelerate bitsandbytes

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 5.2 MB/s  0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 7.9 MB/s eta 0:00:01
   -------------------- ------------------- 0.8/1.5 MB 2.7 MB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.5 MB 1.9 MB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.5 MB 1.9 MB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.5 MB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 1.1 MB/s  0:00:01
   ---------------------------------------- 0.0/59.0 MB ? eta -:--:--
   ---------------------------------------- 0.5/59.0 MB 3.1 MB/s eta 0:00:20
   ---------------------------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install indic-nlp-library

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/3.6 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/3.6 MB 2.7 MB/s eta 0:00:02
   ----------- ---------------------------- 1.0/3.6 MB 3.0 MB/s eta 0:00:01
   -------------------- ------------------- 1.8/3.6 MB 3.4 MB/s eta 0:00:01
   ----------------------------- ---------- 2.6/3.6 MB 3.3 MB/s eta 0:00:01
   ---------------------------------------- 3.6/3.6 MB 3.5 MB/s  0:00:01
   ---------------------------------------- 0.0/587.4 kB ? eta -:--:--
   ---------------------------------------- 587.4/587.4 kB 4.4 MB/s  0:00:00
   ---------------------------------------- 0.0/7.7 MB ? eta -:--:--
   ---- ----------------------------------- 0.8/7.7 MB 3.3 MB/s eta 0:00:03
   ------ --------------------------------- 1.3/7.7 MB 3.4 MB/s eta 0:00:02
   --------- ------------------------------ 1.8/7.7 MB 2.8 MB/s eta 0:00:03
   ------------ ---------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install evaluate

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import re
from typing import List, Dict, Tuple

# Transformers and Hugging Face
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)
from datasets import Dataset, DatasetDict
import evaluate

# PyTorch
import torch.nn as nn
from torch.utils.data import DataLoader


from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [7]:
!git clone https://github.com/AI4Bharat/IndicTrans2.git
!cd IndicTrans2 && pip install -e .

Cloning into 'IndicTrans2'...
fatal: unable to access 'https://github.com/AI4Bharat/IndicTrans2.git/': Could not resolve host: github.com
The system cannot find the path specified.


In [9]:
!git --version

git version 2.47.1.windows.1


In [10]:
!git clone https://github.com/AI4Bharat/IndicTrans2.git
!cd IndicTrans2 && pip install -e .

Cloning into 'IndicTrans2'...


Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///C:/Users/ASUS/Desktop/NLP/Dataset/IndicTrans2



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: file:///C:/Users/ASUS/Desktop/NLP/Dataset/IndicTrans2 does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [2]:
class Config:
    # Paths
    DATA_PATH = "wiki_translate_en_hi_subset.csv"  # Update with your CSV file path
    # OUTPUT_DIR = "./indictrans2_finetuned"
    # CACHE_DIR = "./cache"
    
    # Model
    # MODEL_NAME = "ai4bharat/indictrans2-en-indic-1B"  # or "ai4bharat/indictrans2-en-indic-dist-320M"
    SOURCE_LANG = "eng_Latn"
    TARGET_LANG = "hin_Deva"
    
    # Training parameters
    MAX_LENGTH = 256
    BATCH_SIZE = 8  # Adjust based on GPU memory
    LEARNING_RATE = 2e-5
    NUM_EPOCHS = 3
    WARMUP_STEPS = 500
    WEIGHT_DECAY = 0.01
    
    # Data split
    TEST_SIZE = 0.15
    VAL_SIZE = 0.15
    
    # Hardware
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    FP16 = torch.cuda.is_available()
    
    # Evaluation
    EVAL_STEPS = 500
    SAVE_STEPS = 500
    LOGGING_STEPS = 100

config = Config()

print(f"Using device: {config.DEVICE}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

Using device: cuda
CUDA available: True
GPU: NVIDIA GeForce RTX 2050
GPU Memory: 4.29 GB


In [3]:
def load_data(file_path: str, src_col: str = "src", tgt_col: str = "tgt") -> pd.DataFrame:
    """Load the parallel corpus data"""
    print("Loading data...")
    
    # Try different file formats
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    elif file_path.endswith('.tsv'):
        df = pd.read_csv(file_path, sep='\t')
    elif file_path.endswith('.xlsx'):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format")
    
    # Rename columns if necessary
    if src_col not in df.columns:
        df.columns = ['src', 'tgt']
    
    print(f"Loaded {len(df)} sentence pairs")
    print(f"\nFirst few examples:")
    print(df.head())
    
    return df
df = load_data(config.DATA_PATH, src_col='src', tgt_col='tgt')

Loading data...
Loaded 10000 sentence pairs

First few examples:
                                                 src  \
0  \nBigger (Beyoncé song)\n\n"Bigger" (stylized ...   
1  \n이다\n\nKorean.\nEtymology 2.\nVerb.\nConjugat...   
2  \nHarriet White Fisher\n\nHarriet White Fisher...   
3  \nIris (mythology)\n\nAncient Greek personific...   
4  \nIoannis Kontoyiannis\n\nGreek mathematician ...   

                                                 tgt  
0  \nबड़ा (बेयोंसे गीत)\n\n"बड़ा" (बड़े अक्षर में...  
1  कोरियाई। व्युत्पत्ति 2. क्रिया। संयुग्मन। नोटः...  
2  \nहैरियट सफेद मछुआरा\n\nहैरियट व्हाइट फिशर एंड...  
3  \nआइरिस (पौराणिक कथा)\n\nप्राचीन यूनानी धर्म औ...  
4  \nइओनिस कोंटोयनिस\n\nइओनिस कोंटोयनिस (जन्म जनव...  


In [8]:
# STEP 5: DATA CLEANING AND PREPROCESSING
# ===================================================================
class DataCleaner:
    """Clean and preprocess parallel corpus"""
    
    @staticmethod
    def remove_special_tokens(text: str) -> str:
        """Remove special formatting tokens"""
        if pd.isna(text):
            return ""
        text = re.sub(r'\\n', ' ', str(text))
        text = re.sub(r'\\t', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        return text.strip()
    
    @staticmethod
    def remove_urls(text: str) -> str:
        """Remove URLs from text"""
        return re.sub(r'http\S+|www.\S+', '', text)
    
    @staticmethod
    def normalize_whitespace(text: str) -> str:
        """Normalize whitespace"""
        return ' '.join(text.split())
    
    @staticmethod
    def remove_unwanted_chars(text: str) -> str:
   
        if pd.isna(text):
            return ""
    
        text = str(text)

    # Remove (), /, ., numbers
        text = re.sub(r'[()/\.0-9]', '', text)

    # Normalize extra spaces after removal
        text = re.sub(r'\s+', ' ', text).strip()

        return text

    
    
    @staticmethod
    def is_valid_pair(src: str, tgt: str, 
                     min_length: int = 3, 
                     max_length: int = 500,
                     max_ratio: float = 3.0) -> bool:
        """Check if sentence pair is valid"""
        if not src or not tgt:
            return False
        
        src_len = len(src.split())
        tgt_len = len(tgt.split())
        
        # Check minimum length
        if src_len < min_length or tgt_len < min_length:
            return False
        
        # Check maximum length
        if src_len > max_length or tgt_len > max_length:
            return False
        
        # Check length ratio (avoid very imbalanced pairs)
        ratio = max(src_len, tgt_len) / min(src_len, tgt_len)
        if ratio > max_ratio:
            return False
        
        return True
    
    @classmethod
    def clean_dataframe(cls, df: pd.DataFrame) -> pd.DataFrame:
        """Clean the entire dataframe"""
        print("\nCleaning data...")
        initial_count = len(df)
        
        # Remove duplicates
        df = df.drop_duplicates(subset=['src', 'tgt'])
        print(f"Removed {initial_count - len(df)} duplicates")
        
        # Remove null values
        df = df.dropna(subset=['src', 'tgt'])
        
        # Apply cleaning functions
        df['src'] = df['src'].apply(cls.remove_special_tokens)
        df['tgt'] = df['tgt'].apply(cls.remove_special_tokens)
        
        df['src'] = df['src'].apply(cls.remove_urls)
        df['tgt'] = df['tgt'].apply(cls.remove_urls)
        
        df['src'] = df['src'].apply(cls.normalize_whitespace)
        df['tgt'] = df['tgt'].apply(cls.normalize_whitespace)
        
        # Filter valid pairs
        mask = df.apply(lambda row: cls.is_valid_pair(row['src'], row['tgt']), axis=1)
        df = df[mask].reset_index(drop=True)
        
        print(f"Final dataset size: {len(df)} sentence pairs")
        print(f"Removed {initial_count - len(df)} invalid pairs")
        
        return df

# Clean the data
df_clean = DataCleaner.clean_dataframe(df)

# Display statistics
print("\n=== Data Statistics ===")
print(f"Source language word count: {df_clean['src'].str.split().str.len().describe()}")
print(f"\nTarget language word count: {df_clean['tgt'].str.split().str.len().describe()}")


Cleaning data...
Removed 0 duplicates
Final dataset size: 6654 sentence pairs
Removed 3346 invalid pairs

=== Data Statistics ===
Source language word count: count    6654.000000
mean      171.306733
std       120.653526
min         3.000000
25%        74.250000
50%       144.000000
75%       258.000000
max       497.000000
Name: src, dtype: float64

Target language word count: count    6654.000000
mean      182.836339
std       133.439584
min         4.000000
25%        74.000000
50%       151.000000
75%       278.000000
max       500.000000
Name: tgt, dtype: float64


In [9]:
print(df_clean)

                                                    src  \
0     Bigger (Beyoncé song) "Bigger" (stylized in up...   
1     이다 Korean. Etymology 2. Verb. Conjugation. Not...   
2     Ioannis Kontoyiannis Greek mathematician and i...   
3     Calgary Roller Derby Roller derby league Calga...   
4     Cork North (Dáil constituency) Dáil constituen...   
...                                                 ...   
6649  Inferior thyroid veins The inferior thyroid ve...   
6650  Troyebratka Troyebratka () is a rural locality...   
6651  اجلماك Karakhanid. Etymology. From . Equivalen...   
6652  Kingswood Luas stop Tram stop in Dublin, Irela...   
6653  Ante-Nicene Fathers/Volume IV/Commodianus/The ...   

                                                    tgt  
0     बड़ा (बेयोंसे गीत) "बड़ा" (बड़े अक्षर में शैली...  
1     कोरियाई। व्युत्पत्ति 2. क्रिया। संयुग्मन। नोटः...  
2     इओनिस कोंटोयनिस इओनिस कोंटोयनिस (जन्म जनवरी 19...  
3     कैलगरी रोलर डर्बी कैलगरी रोलर डर्बी एक गैर-लाभ...  
4

In [10]:
dff = df.head(1)
print(dff.to_string())